# [Selenium](https://selenium-python.readthedocs.io/installation.html)
* You have to download its driver which browser you are working on. 
* Since we are working on Chrome, we are downloading the correct version of the [Chrome driver](https://sites.google.com/a/chromium.org/chromedriver/downloads).
* To check the chrome version of your Operating Sytem, [click here](https://chromedriver.storage.googleapis.com/LATEST_RELEASE)

### eksisozluk
https://eksisozluk.com/corona-virusu-sayesinde-fark-edilen-gercekler--6435737?p=

In [4]:
# pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

PATH = "C:\Program Files (x86)\chromedriver.exe"
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(chrome_options=options, executable_path=PATH)

url = """https://eksisozluk.com/corona-virusu-sayesinde-fark-edilen-gercekler--6435737?p="""

pageCount = 115
df = pd.DataFrame(index=range(0,pageCount*10),columns=['entries'])
index=0

for i in range(1,pageCount+1):
    driver.get(url+str(i))
    elements = driver.find_elements_by_class_name('content')
    for i in elements:
        df['entries'][index]=i.text
        index += 1
    
driver.close()

df.dropna(axis=0,inplace=True)

# Create a txt file with entries
with open("Data/eksisozluk_entries.txt", "w", encoding="UTF-8") as file:
    for i in range(len(df)):
        file.write(f"""{i+1}.\n{df.iloc[i,0]}\n{'-'*50}\n""")

display(df)

,entries
0,"hayat gailesi içinde kaybolmuşken, corona duva..."
1,ihtilal ve yokluk görmüş bir neslin şu an gözl...
2,modern hayatta birçok kişinin dışardan hizmet ...
3,her şeye muktedir olduğunu sanan kibirli insan...
4,bu ülkede 50 yaş üzerine gerçekten laf anlatma...
...,...
1141,"erteleme,istediğin şeyleri yap,ölüm çok yakın\..."
1142,çevrenizdeki dostların ne kadar dost olduğu.
1143,eski sıradan hayatımızın aslında ne kadar müke...
1144,evden çalışarak da çoğu işin halledilebildiği....


### imdb
https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating

In [28]:
# pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
import numpy as np
import time


def get_movies(num_movies, path):
    
    #Initializing the webdriver
    options = Options()
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(chrome_options=options, executable_path=path)

    driver.set_window_size(1120, 1000)

    #Open Specified URL with keyword you want to search for
    URL = "https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"
    driver.get(URL)
    
    movies = []
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    

    while len(movies) < num_movies:
        
        try:
            #Wait until the web page load
            element = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
                    EC.presence_of_element_located((By.ID, "main"))
                    )
        
            movie_button = element.find_elements_by_xpath('.//div[@class="lister-item mode-advanced"]')

            for movie in movie_button:

                print("Progress: {}".format("" + str(len(movies)) + "/" + str(num_movies)))
                if len(movies) >= num_movies:
                    break

                try:
                    title = movie.find_element_by_xpath('.//h3[@class="lister-item-header"]').text.split('.')[1].strip()
                except NoSuchElementException:
                    title = np.NaN
                
                try: 
                    certificate = movie.find_element_by_xpath('.//span[@class="certificate"]').text
                except NoSuchElementException:
                    certificate = np.NaN
                    
                try:    
                    duration = movie.find_element_by_xpath('.//span[@class="runtime"]').text
                except NoSuchElementException:
                    duration = np.NAN
                    
                try:
                    genre = movie.find_element_by_xpath('.//span[@class="genre"]').text
                except NoSuchElementException:
                    genre = np.NaN
                    
                try:
                    rate = movie.find_element_by_xpath('.//strong').text 
                except NoSuchElementException:
                    rate = np.NaN
                
                try: 
                    metascore = movie.find_element_by_xpath('.//span[@class="metascore  favorable"]').text
                except NoSuchElementException:
                    metascore = np.NaN
                    
                try:
                    descr = movie.find_element_by_xpath('.//p[@class="text-muted"]').text
                except NoSuchElementException:
                    descr = np.NaN
                
                try:
                    cast = movie.find_element_by_xpath('.//p[@class=""]').text
                except NoSuchElementException:
                    cast = np.NaN
                
                try:
                    info = movie.find_element_by_xpath('.//p[@class="sort-num_votes-visible"]').text
                except NoSuchElementException:
                    info = np.NaN
                
                movies.append({
                    "Title": title,
                    "Certificate": certificate,
                    "Duration": duration,
                    "Genre": genre,
                    "Rate": rate,
                    "Metascore": metascore,
                    "Description": descr,
                    "Cast": cast,
                    "Info": info
                    })
            try:
                if len(movies) < num_movies:
                    element.find_element_by_xpath('.//a[@class="lister-page-next next-page"]').click()
                    time.sleep(20)
                
            except NoSuchElementException:
                print("Scraping terminated before reaching target number of movies. Needed {}, got {}.".format(num_movies, len(movies)))
                driver.quit()
                break   
            
        finally:
            if len(movies) >= num_movies:
                driver.quit()

    return pd.DataFrame(data=movies)

PATH = "C:\Program Files (x86)\chromedriver.exe"
df = get_movies(1000, PATH)

# Create a txt file with entries
with open("Data/imdb_data.txt", "w", encoding="UTF-8") as file:
    for k,v in enumerate(df.to_dict('record')):
        file.write(f"{k+1}.\n\n")
        for i,j in v.items():
            file.write(f"{i:<15} : {j}\n")
        file.write(f'\n{"*"*70}\n')
        
df.to_csv("Data/IMDB top 1000.csv")
display(df)
        
# for k,v in enumerate(df.to_dict('record')):
#     print(k,'.','\n')
#     for i,j in v.items():
#         print(f"{i:<15} : {j}")
#     print('\n',"*"*70,'\n')

<ipython-input-28-02096841c523>:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=path)


Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
Progress: 30/1000
Progress: 31/1000
Progress: 32/1000
Progress: 33/1000
Progress: 34/1000
Progress: 35/1000
Progress: 36/1000
Progress: 37/1000
Progress: 38/1000
Progress: 39/1000
Progress: 40/1000
Progress: 41/1000
Progress: 42/1000
Progress: 43/1000
Progress: 44/1000
Progress: 45/1000
Progress: 46/1000
Progress: 47/1000
Progress: 48/1000
Progress: 49/1000
Progress: 50/1000
Progress: 51/1000
Progress: 52/1000
Progress: 53/1000
Progress: 54/1000
Progress: 55/1000
Pr

Progress: 437/1000
Progress: 438/1000
Progress: 439/1000
Progress: 440/1000
Progress: 441/1000
Progress: 442/1000
Progress: 443/1000
Progress: 444/1000
Progress: 445/1000
Progress: 446/1000
Progress: 447/1000
Progress: 448/1000
Progress: 449/1000
Progress: 450/1000
Progress: 451/1000
Progress: 452/1000
Progress: 453/1000
Progress: 454/1000
Progress: 455/1000
Progress: 456/1000
Progress: 457/1000
Progress: 458/1000
Progress: 459/1000
Progress: 460/1000
Progress: 461/1000
Progress: 462/1000
Progress: 463/1000
Progress: 464/1000
Progress: 465/1000
Progress: 466/1000
Progress: 467/1000
Progress: 468/1000
Progress: 469/1000
Progress: 470/1000
Progress: 471/1000
Progress: 472/1000
Progress: 473/1000
Progress: 474/1000
Progress: 475/1000
Progress: 476/1000
Progress: 477/1000
Progress: 478/1000
Progress: 479/1000
Progress: 480/1000
Progress: 481/1000
Progress: 482/1000
Progress: 483/1000
Progress: 484/1000
Progress: 485/1000
Progress: 486/1000
Progress: 487/1000
Progress: 488/1000
Progress: 48

Progress: 869/1000
Progress: 870/1000
Progress: 871/1000
Progress: 872/1000
Progress: 873/1000
Progress: 874/1000
Progress: 875/1000
Progress: 876/1000
Progress: 877/1000
Progress: 878/1000
Progress: 879/1000
Progress: 880/1000
Progress: 881/1000
Progress: 882/1000
Progress: 883/1000
Progress: 884/1000
Progress: 885/1000
Progress: 886/1000
Progress: 887/1000
Progress: 888/1000
Progress: 889/1000
Progress: 890/1000
Progress: 891/1000
Progress: 892/1000
Progress: 893/1000
Progress: 894/1000
Progress: 895/1000
Progress: 896/1000
Progress: 897/1000
Progress: 898/1000
Progress: 899/1000
Progress: 900/1000
Progress: 901/1000
Progress: 902/1000
Progress: 903/1000
Progress: 904/1000
Progress: 905/1000
Progress: 906/1000
Progress: 907/1000
Progress: 908/1000
Progress: 909/1000
Progress: 910/1000
Progress: 911/1000
Progress: 912/1000
Progress: 913/1000
Progress: 914/1000
Progress: 915/1000
Progress: 916/1000
Progress: 917/1000
Progress: 918/1000
Progress: 919/1000
Progress: 920/1000
Progress: 92

,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info
0,Esaretin Bedeli (1994),13+,142 min,Drama,"9,3",80,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,...",Votes: 2.381.013 | Gross: $28.34M
1,Baba (1972),16,175 min,"Crime, Drama","9,2",100,An organized crime dynasty's aging patriarch t...,Director: Francis Ford Coppola | Stars: Marlon...,Votes: 1.649.813 | Gross: $134.97M
2,Kara Şövalye (2008),16,152 min,"Action, Crime, Drama","9,0",84,When the menace known as the Joker wreaks havo...,Director: Christopher Nolan | Stars: Christian...,Votes: 2.344.801 | Gross: $534.86M
3,Baba 2 (1974),12,202 min,"Crime, Drama","9,0",90,The early life and career of Vito Corleone in ...,Director: Francis Ford Coppola | Stars: Al Pac...,Votes: 1.148.381 | Gross: $57.30M
4,12 Öfkeli Adam (1957),14,96 min,"Crime, Drama","9,0",96,A jury holdout attempts to prevent a miscarria...,"Director: Sidney Lumet | Stars: Henry Fonda, L...",Votes: 703.044 | Gross: $4.36M
...,...,...,...,...,...,...,...,...,...
995,Altına hücum (1925),NaN,95 min,"Adventure, Comedy, Drama","8,2",NaN,A prospector goes to the Klondike in search of...,Director: Charles Chaplin | Stars: Charles Cha...,Votes: 102.823 | Gross: $5.45M
996,Sherlock Jr,NaN,45 min,"Action, Comedy, Romance","8,2",NaN,"A film projectionist longs to be a detective, ...",Director: Buster Keaton | Stars: Buster Keaton...,Votes: 43.315 | Gross: $0.98M
997,Alev Almış Bir Genç Kızın Portresi (2019),12,122 min,"Drama, Romance","8,1",95,On an isolated island in Brittany at the end o...,Director: Céline Sciamma | Stars: Noémie Merla...,Votes: 69.625 | Gross: $3.76M
998,Pink (III) (2016),12,136 min,"Drama, Thriller","8,1",NaN,When three young women are implicated in a cri...,Director: Aniruddha Roy Chowdhury | Stars: Taa...,Votes: 40.025 | Gross: $1.24M


### sofifa
https://sofifa.com/

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium. common. exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
PATH = "C:\Program Files (x86)\chromedriver.exe"
listem=[]
sy=1
while sy<30000:
    browser=webdriver.Chrome(executable_path=PATH,options=chrome_options) 
    browser.get("https://sofifa.com/?offset="+str(sy)+"")
    n=1
    time.sleep(0.2)
    while n<61:
        browser=webdriver.Chrome(executable_path=PATH,options=chrome_options) 
        browser.get("https://sofifa.com/?offset="+str(sy)+"")
       
        gir=browser.find_element_by_xpath("//*[@id='adjust']/div/div[1]/div/table/tbody/tr["+str(n)+"]/td[2]/a[1]/div")
        
        time.sleep(1)
        gir.click()
        
        try:
            ismi=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[1]/div[1]/div[1]/div/div/h1").text
            print(ismi)
            pozisyon=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[2]/div[1]/ul/li[1]/span").text
            puanı=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[1]/div[1]/div[1]/div/section/div/div[1]/div/span").text
            değeri=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[1]/div[1]/div[1]/div/section/div/div[3]/div").text
            maaşı=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[1]/div[1]/div[1]/div/section/div/div[4]/div").text
            serbest_kalma_fiyatı=browser.find_element_by_xpath("//*[@id='list']/div[2]/div/div/div[1]/div[1]/div[2]/div/div[1]/div/ul/li[8]/span").text
        except:
            pass
        try:
            yine_gir=browser.find_element_by_xpath("//*[@id='list']/header/div[3]/div/div/div/a[2]")
        except:
            pass
        time.sleep(0.3)
        yine_gir.click()
        
        time.sleep(1)
       
        
        try:
            yaşı=browser.find_element_by_xpath("//*[@id='list']/div[1]/div/div/div[2]/div[1]/ul[1]/li[1]").text
        except:
            yaşı="nan"
        try:
            kilo=browser.find_element_by_xpath("//*[@id='list']/div[1]/div/div/div[2]/div[1]/ul[1]/li[3]").text
        except:
            kilo="nan"
        try:
            ülke=browser.find_element_by_xpath("//*[@id='list']/div[1]/div/div/div[2]/div[1]/ul[2]/li[1]/a").get_attribute("title")
        except:
            ülke="nan"
        time.sleep(0.3)
       
        x=1
        bilgileri=[]
        while x<10:
            
            bunda=browser.find_elements_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]")
            
            for i in bunda:
                sezon=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[1]").text
                takım=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[2]").text
                lig=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[3]").text
                oynadıgı_süre=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[4]").text
                appearance=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[5]").text
                lineup=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[6]").text
                substitute_in=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[7]").text
                substitute_out=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[8]").text
                substitute_on_bench=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[9]").text
                goal=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[10]").text
                yellow=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[10]").text
                yellow_2nd=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[10]").text
                red=i.find_element_by_xpath("//*[@id='page_player_1_block_player_career_6_table']/tbody/tr["+str(x)+"]/td[10]").text
                bilgileri.append([sezon,takım,lig,lineup,oynadıgı_süre,appearance,lineup,substitute_in,substitute_out,substitute_on_bench,goal,yellow,yellow_2nd,red])
            
            x+=1
           
        browser.quit()
        
        n+=1
        listem.append([ismi,pozisyon,ülke,puanı,değeri,maaşı,serbest_kalma_fiyatı,yaşı,kilo,bilgileri])
        print(listem)
    sy+=60   



Mason Mount
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]]]
Édouard Mendy
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7'

Philip Foden
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['201

Florian Thauvin
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['

Reece James
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['2019

Thiago Emiliano da Silva
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0',

Hatem Ben Arfa
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['2

[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['2019/2020', 'Ren

[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['2019/2020', 'Ren

Riyad Mahrez
[['Mason Mount', 'CAM', 'England', '81', '€41.5M\nValue', '€75K\nWage', '€87.2M', '22 y.o.', '5\'10" 154lbs', [['2020/2021', 'Chelsea', 'PRL', '29', '2635', '33', '29', '4', '5', '5', '6', '6', '6', '6'], ['2019/2020', 'Chelsea', 'PRL', '32', '2874', '37', '32', '5', '12', '6', '7', '7', '7', '7'], ['2018/2019', 'Derby County', 'CHA', '38', '3321', '38', '38', '0', '7', '0', '9', '9', '9', '9'], ['2017/2018', 'Vitesse', 'ERE', '25', '2428', '32', '25', '7', '2', '12', '14', '14', '14', '14'], ['2016/2017', 'Chelsea U18', 'UPL', '4', '360', '4', '4', '0', '0', '0', '1', '1', '1', '1'], ['2016/2017', 'Chelsea U23', 'PL2', '19', '1687', '21', '19', '2', '5', '0', '4', '4', '4', '4']]], ['Édouard Mendy', 'GK', 'Senegal', '80', '€18M\nValue', '€74K\nWage', '€35.6M', '29 y.o.', '6\'6" 190lbs', [['2020/2021', 'Chelsea', 'PRL', '27', '2430', '27', '27', '0', '0', '2', '0', '0', '0', '0'], ['2020/2021', 'Rennes', 'LI1', '1', '90', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['201

KeyboardInterrupt: 